<a href="https://colab.research.google.com/github/ROHITPL24/Predictive-Analysis-for-Manufacturing-Operations/blob/main/main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn pandas numpy scikit-learn python-multipart nest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.6 MB/s eta 0:00:00


In [25]:
!pip install matplotlib-venn


In [26]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [27]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 124565 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.3_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.3) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.3) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.0 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31631 sha256=c2531406601912a226f1d36b218cb2158b2ffe72f9edd5117aa4de3b96fc75e9
  Stored in directory: /root/.cache/pip/wheels/32/98/bd/4893d6923dd027f455b250367d402bfd69a6f4416581df46db
Successfully built libarchive


In [28]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [29]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 82.2 MB/s eta 0:00:00


In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

# Generate synthetic data
def generate_sample_data(n_samples=1000):
    np.random.seed(42)
    data = {
        'Machine_ID': np.random.randint(1, 11, n_samples),
        'Temperature': np.random.normal(75, 15, n_samples),
        'Run_Time': np.random.normal(100, 25, n_samples)
    }
    df = pd.DataFrame(data)
    df['Downtime_Flag'] = ((df['Temperature'] > 90) |
                          (df['Run_Time'] > 140)).astype(int)
    return df

# Generate data
df = generate_sample_data()
print("Data Shape:", df.shape)
df.head()


Data Shape: (1000, 4)


,Machine_ID,Temperature,Run_Time,Downtime_Flag
0,7,88.017736,140.816312,1
1,4,60.952290,93.477193,0
2,8,61.867125,123.612274,0
3,5,78.069153,82.769273,0
4,7,65.960283,92.438329,0


In [34]:
# Create interactive scatter plot
fig = px.scatter(df, x='Temperature', y='Run_Time',
                color='Downtime_Flag',
                title='Temperature vs Run Time (Colored by Downtime)',
                color_discrete_sequence=['blue', 'red'],
                labels={'Downtime_Flag': 'Downtime'},
                hover_data=['Machine_ID'])

fig.update_layout(width=800, height=600)
fig.show()


In [35]:
# Create distribution plots
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Temperature Distribution by Downtime',
                                   'Run Time Distribution by Downtime',
                                   'Downtime Distribution',
                                   'Machine ID Distribution'))

# Temperature distribution
for downtime in [0, 1]:
    fig.add_trace(
        go.Histogram(x=df[df['Downtime_Flag']==downtime]['Temperature'],
                     name=f'Downtime={downtime}',
                     opacity=0.7),
        row=1, col=1
    )

# Run Time distribution
for downtime in [0, 1]:
    fig.add_trace(
        go.Histogram(x=df[df['Downtime_Flag']==downtime]['Run_Time'],
                     name=f'Downtime={downtime}',
                     opacity=0.7),
        row=1, col=2
    )

# Downtime distribution
downtime_counts = df['Downtime_Flag'].value_counts()
fig.add_trace(
    go.Bar(x=['No Downtime', 'Downtime'],
           y=downtime_counts.values),
    row=2, col=1
)

# Machine ID distribution
machine_counts = df['Machine_ID'].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=machine_counts.index,
           y=machine_counts.values),
    row=2, col=2
)

fig.update_layout(height=800, width=1000, showlegend=False)
fig.show()


In [36]:
# Train the model and visualize results
# Prepare features and target
X = df[['Temperature', 'Run_Time']]
y = df['Downtime_Flag']

# Split and scale data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
fig = px.imshow(cm,
                labels=dict(x="Predicted", y="Actual"),
                x=['No Downtime', 'Downtime'],
                y=['No Downtime', 'Downtime'],
                title='Confusion Matrix',
                color_continuous_scale='RdBu')

# Add text annotations to confusion matrix
for i in range(len(cm)):
    for j in range(len(cm[i])):
        fig.add_annotation(x=j, y=i,
                          text=str(cm[i][j]),
                          showarrow=False,
                          font=dict(color='white'))

fig.show()

print(f"\nModel Performance:")
print(f"Accuracy: {accuracy:.3f}")
print(f"F1 Score: {f1:.3f}")



Model Performance:
Accuracy: 1.000
F1 Score: 1.000


In [37]:
# Function to make predictions with visualization
def predict_and_visualize(temperature, run_time):
    # Prepare input data
    input_data = pd.DataFrame({
        'Temperature': [temperature],
        'Run_Time': [run_time]
    })

    # Scale input
    input_scaled = scaler.transform(input_data)

    # Make prediction
    prediction = model.predict(input_scaled)[0]
    confidence = max(model.predict_proba(input_scaled)[0])

    # Create visualization
    fig = px.scatter(df, x='Temperature', y='Run_Time',
                    color='Downtime_Flag',
                    title='Prediction Visualization',
                    color_discrete_sequence=['blue', 'red'])

    # Add the prediction point
    fig.add_trace(
        go.Scatter(x=[temperature], y=[run_time],
                  mode='markers',
                  marker=dict(size=15, symbol='star',
                             color='yellow', line=dict(width=2)),
                  name='New Prediction')
    )

    fig.show()

    return {
        "Downtime": "Yes" if prediction == 1 else "No",
        "Confidence": round(float(confidence), 3)
    }

# Test cases
test_cases = [
    (95, 150),  # High risk case
    (70, 90)    # Low risk case
]

for temp, run_time in test_cases:
    print(f"\nPrediction for Temperature={temp}, Run_Time={run_time}:")
    result = predict_and_visualize(temp, run_time)
    print(f"Result: {result}")



Prediction for Temperature=95, Run_Time=150:


Result: {'Downtime': 'Yes', 'Confidence': 1.0}

Prediction for Temperature=70, Run_Time=90:


Result: {'Downtime': 'No', 'Confidence': 1.0}
